In [66]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from TextCleaner import TextCleaner
from helpers import binary_confusion_matrix
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns
import scikitplot as skplt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
import tensorflow_hub as hub
import pandas_profiling as pr
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
%matplotlib inline

@inproceedings{wang2018eann,
  title={EANN: Event Adversarial Neural Networks for Multi-Modal Fake News Detection},
  author={Wang, Yaqing and Ma, Fenglong and Jin, Zhiwei and Yuan, Ye and Xun, Guangxu and Jha, Kishlay and Su, Lu and Gao, Jing},
  booktitle={Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining},
  pages={849--857},
  year={2018},
  organization={ACM}
}

https://arxiv.org/abs/1803.11175

In [73]:
df = pd.read_csv('data/train.csv')
df = df.loc[(~df.text.isnull()) | (~df.title.isnull())]
df.fillna('MISSING', inplace=True)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [74]:
pr.ProfileReport(df)

Number of variables,5
Number of observations,20800
Total Missing (%),0.0%
Total size in memory,975.0 KiB
Average record size in memory,48.0 B
Numeric,1
Categorical,3
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [75]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [76]:
df.shape

(20800, 5)

In [77]:
# combine columnt texts
df['combined_text'] = df[['title', 'author', 'text']].apply(lambda x: ''.join(str(x)), axis=1)
df.head()

,id,title,author,text,label,combined_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,title House Dem Aide: We Didn’t Even See C...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"title FLYNN: Hillary Clinton, Big Woman on..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,title Why the Truth Might ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,title 15 Civilians Killed In Single US Air...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,title Iranian woman jailed for fictional u...


In [ ]:
# # combine columnt texts
# df['combined_text'] = TextCleaner(custom_stopwords=['title', 'author', 'text']).transform(df['combined_text'])
# df.head()

In [94]:
#df['title'] = TextCleaner().transform(df['title'].astype(str))
txt = df['title'].str.split().astype(str).tolist() #combined_text got 81% accuracy
txt[:5]

["['House', 'Dem', 'Aide:', 'We', 'Didn’t', 'Even', 'See', 'Comey’s', 'Letter', 'Until', 'Jason', 'Chaffetz', 'Tweeted', 'It']",
 "['FLYNN:', 'Hillary', 'Clinton,', 'Big', 'Woman', 'on', 'Campus', '-', 'Breitbart']",
 "['Why', 'the', 'Truth', 'Might', 'Get', 'You', 'Fired']",
 "['15', 'Civilians', 'Killed', 'In', 'Single', 'US', 'Airstrike', 'Have', 'Been', 'Identified']",
 "['Iranian', 'woman', 'jailed', 'for', 'fictional', 'unpublished', 'story', 'about', 'woman', 'stoned', 'to', 'death', 'for', 'adultery']"]

In [30]:
# from nltk.tag import UnigramTagger, DefaultTagger
# from nltk.corpus import treebank
# tagger = DefaultTagger('NN')
# #tagger.tag(txt)

# Universal Sentence Embeddings

In [48]:
# import tensorflow as tf
# import tensorflow_hub as hub

def fetch_universal_sentence_embeddings(messages, verbose=0):
    """Fetches universal sentence embeddings from Google's
    research paper https://arxiv.org/pdf/1803.11175.pdf.
    
    INPUTS:
    RETURNS:
    """
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

    # Import the Universal Sentence Encoder's TF Hub module
    embed = hub.Module(module_url)

    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(messages))
        embeddings = list()
        for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
            if verbose:
                print("Message: {}".format(messages[i]))
                print("Embedding size: {}".format(len(message_embedding)))
                message_embedding_snippet = ", ".join(
                    (str(x) for x in message_embedding[:3]))
                print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
            embeddings.append(message_embedding)
    return embeddings

embeddings = fetch_universal_sentence_embeddings(txt)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [49]:
embeddings[0][:10]

[0.05387789383530617,
 0.006952833849936724,
 -0.057719238102436066,
 0.016467709094285965,
 -0.08529816567897797,
 0.007254021242260933,
 0.08881591260433197,
 0.007081518415361643,
 -0.03469545766711235,
 -0.0006816966342739761]

In [50]:
df.label.value_counts() / df.shape[0]

1    0.500625
0    0.499375
Name: label, dtype: float64

In [51]:
from sklearn.model_selection import train_test_split
X, y = np.array(embeddings), df.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=7)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=.5, random_state=7)

In [52]:
X[0][:5]

array([ 0.05387789,  0.00695283, -0.05771924,  0.01646771, -0.08529817])

# Baseline Model

In [20]:
# test LinearSVC model as baseline

params = {'clf__C':[1, 2, 3, 4, 5], # 1 best here  [.001, .01, .1, 1, 10, 100, 1000]
          'clf__random_state': [777]}

pipeline = Pipeline([
    ('clf', LinearSVC())
])

grid = GridSearchCV(pipeline, params, cv=5, verbose=1)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   38.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': [1, 2, 3, 4, 5], 'clf__random_state': [777]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [21]:
grid.best_estimator_.get_params()

{'memory': None,
 'steps': [('clf',
   LinearSVC(C=2, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=777, tol=0.0001,
        verbose=0))],
 'clf': LinearSVC(C=2, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=777, tol=0.0001,
      verbose=0),
 'clf__C': 2,
 'clf__class_weight': None,
 'clf__dual': True,
 'clf__fit_intercept': True,
 'clf__intercept_scaling': 1,
 'clf__loss': 'squared_hinge',
 'clf__max_iter': 1000,
 'clf__multi_class': 'ovr',
 'clf__penalty': 'l2',
 'clf__random_state': 777,
 'clf__tol': 0.0001,
 'clf__verbose': 0}

In [22]:
def binary_confusion_matrix(y, y_hat, as_pct=False, verbose=True):
    cm = pd.DataFrame(confusion_matrix(y, y_hat),
                      columns=['(+) actual', '(-) actual'],
                      index=['(+) predicted', '(-) predicted'])
    if as_pct:
        cm = cm / cm.sum().sum()

    P = cm['(+) actual'].sum()
    N = cm['(-) actual'].sum()
    total = P + N
    TP = cm.loc['(+) predicted', '(+) actual']
    FP = cm.loc['(+) predicted', '(-) actual']
    TN = cm.loc['(-) predicted', '(-) actual']
    FN = cm.loc['(-) predicted', '(+) actual']
    TPR = TP / (TP + FN)          # recall/sensitivity
    TNR = TN / (TN + FP)   # specificity
    FPR = FP / (FP + TN)   # fall-out
    FNR = FN / (FN + TP)   # miss rate
    PPV = TP / (TP + FP)   # precision
    NPV = TN / (TN + FN)   # neg predictive value

    if verbose:
        print('''
        Condition Positive:                        %i
        Condition Negative:                        %i
        Total Observations:                        %i

        True Positive:                             %i
        True Negative:                             %i
        False Positive:                            %i
        False Negative                             %i

        True Positive Rate (recall):               %.2f%%
        True Negative Rate (specificity):          %.2f%%
        False Positive Rate (fall-out):            %.2f%%
        False Negative Rate (miss rate):           %.2f%%

        Positive Predictive Value (precision):     %.2f%%
        Negative Predictive Value:                 %.2f%%
        ''' %(P, N, total,
             TP, TN, FP, FN,
             TPR*100, TNR*100, FPR*100, FNR*100,
             PPV*100, NPV*100))

    metrics = {'P': P, 'N': N, 'total': total,
              'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
              'TPR': TPR, 'TNR': TNR, 'FPR': FPR, 'FNR': FNR, 'PPV': PPV, 'NPV': NPV}

    return cm, metrics


yhat_val = grid.predict(X_val)
val_acc = accuracy_score(y_val, yhat_val)
yhat_train = grid.predict(X_train)
train_acc = accuracy_score(y_train, yhat_train) 

print(f'''
Training Accuracy: {round(train_acc, 4)}
Validation Accuracy: {round(val_acc, 4)}
''')

_, __ = binary_confusion_matrix(y_val, yhat_val)
_


Training Accuracy: 0.8994
Validation Accuracy: 0.8875


        Condition Positive:                        1511
        Condition Negative:                        1609
        Total Observations:                        3120

        True Positive:                             1340
        True Negative:                             1429
        False Positive:                            180
        False Negative                             171

        True Positive Rate (recall):               88.68%
        True Negative Rate (specificity):          88.81%
        False Positive Rate (fall-out):            11.19%
        False Negative Rate (miss rate):           11.32%

        Positive Predictive Value (precision):     88.16%
        Negative Predictive Value:                 89.31%
        


,(+) actual,(-) actual
(+) predicted,1340,180
(-) predicted,171,1429


In [23]:
help(LinearSVCarSVC)

Help on class LinearSVC in module sklearn.svm.classes:

class LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.linear_model.base.SparseCoefMixin)
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  penalty : string, 'l1' or 'l2' (default='l2')
 |      Specifies the norm used in the penalization. The 'l2'
 |      penalty is the standard used in SVC. The 'l1' leads to ``coef_``
 |      vectors that are sparse.
 |  
 |  loss : string, 'hinge' or 'squared_hinge' (defau

In [ ]:
import scikitplot as skplt
import matplotlib as plt
%matplotlib inline
skplt.metrics.plot_roc_curve(y_val, grid.predict_proba(X_val))

# Test Autoencoder to Enhance Feature Set of Text

Titles of news articles worked well -- now let's combine the power of sentence embeddings on short sequences (titles) with the power of autoencoders on the entire cleaned text.

In [36]:
help(Embedding)

Help on class Embedding in module keras.layers.embeddings:

class Embedding(keras.engine.topology.Layer)
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  # Example
 |  
 |  ```python
 |    model = Sequential()
 |    model.add(Embedding(1000, 64, input_length=10))
 |    # the model will take as input an integer matrix of size (batch, input_length).
 |    # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
 |    # now model.output_shape == (None, 10, 64), where None is the batch dimension.
 |  
 |    input_array = np.random.randint(1000, size=(32, 10))
 |  
 |    model.compile('rmsprop', 'mse')
 |    output_array = model.predict(input_array)
 |    assert output_array.shape == (32, 10, 64)
 |  ```
 |  
 |  # Arguments
 |    input_dim: int > 0. Size of the vocabulary,
 |        i.e. maximum int

In [53]:
X_train.shape

(14559, 512)

In [60]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

- column selector, feature union
    - autoencoder of text in article
        - Transform text via embeddings
    - model on title using sentence embeddings

In [64]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[3, 40], [36, 29], [36, 16], [23, 29], [38], [5], [26, 16], [10, 36], [26, 29], [30, 7, 40, 4]]
[[ 3 40  0  0]
 [36 29  0  0]
 [36 16  0  0]
 [23 29  0  0]
 [38  0  0  0]
 [ 5  0  0  0]
 [26 16  0  0]
 [10 36  0  0]
 [26 29  0  0]
 [30  7 40  4]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_105 (Dense)            (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 69.999999


In [82]:
X_embedded.shape

(14559, 512, 64)

In [85]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
help(pad_sequences)

Help on function pad_sequences in module keras.preprocessing.sequence:

pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
    Pads sequences to the same length.
    
    This function transforms a list of
    `num_samples` sequences (lists of integers)
    into a 2D Numpy array of shape `(num_samples, num_timesteps)`.
    `num_timesteps` is either the `maxlen` argument if provided,
    or the length of the longest sequence otherwise.
    
    Sequences that are shorter than `num_timesteps`
    are padded with `value` at the end.
    
    Sequences longer than `num_timesteps` are truncated
    so that they fit the desired length.
    The position where padding or truncation happens is determined by
    the arguments `padding` and `truncating`, respectively.
    
    Pre-padding is the default.
    
    # Arguments
        sequences: List of lists, where each element is a sequence.
        maxlen: Int, maximum length of all sequences.
      

In [105]:
txt = df['text'].str.split().astype(str)
txt[0]

'[\'House\', \'Dem\', \'Aide:\', \'We\', \'Didn’t\', \'Even\', \'See\', \'Comey’s\', \'Letter\', \'Until\', \'Jason\', \'Chaffetz\', \'Tweeted\', \'It\', \'By\', \'Darrell\', \'Lucus\', \'on\', \'October\', \'30,\', \'2016\', \'Subscribe\', \'Jason\', \'Chaffetz\', \'on\', \'the\', \'stump\', \'in\', \'American\', \'Fork,\', \'Utah\', \'(\', \'image\', \'courtesy\', \'Michael\', \'Jolley,\', \'available\', \'under\', \'a\', \'Creative\', \'Commons-BY\', \'license)\', \'With\', \'apologies\', \'to\', \'Keith\', \'Olbermann,\', \'there\', \'is\', \'no\', \'doubt\', \'who\', \'the\', \'Worst\', \'Person\', \'in\', \'The\', \'World\', \'is\', \'this\', \'week–FBI\', \'Director\', \'James\', \'Comey.\', \'But\', \'according\', \'to\', \'a\', \'House\', \'Democratic\', \'aide,\', \'it\', \'looks\', \'like\', \'we\', \'also\', \'know\', \'who\', \'the\', \'second-worst\', \'person\', \'is\', \'as\', \'well.\', \'It\', \'turns\', \'out\', \'that\', \'when\', \'Comey\', \'sent\', \'his\', \'now

In [107]:
# resplit data but ignore y
X, y = np.array(txt), df.label.values
txt_train, txt_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=7)

num_words = 5000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(txt)

X_train = tokenizer.texts_to_sequences(txt_train)
X_test = tokenizer.texts_to_sequences(txt_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(txt_train[2][:50])
print(X_train[2][:50])

['WASHINGTON', '—', 'Reduced', 'to', 'their', 'wea
[256, 35, 3331, 3, 39, 100, 7, 6, 1, 241, 223, 511, 43, 7, 88, 1277, 28, 1808, 3, 3756, 1669, 5, 4008, 6, 58, 137, 992, 12, 2, 2735, 4, 2, 1980, 2450, 1, 30, 517, 241, 183, 8, 2, 27, 245, 46, 2997, 10, 89, 35, 18, 29]


In [108]:
maxlen = 5000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [110]:
X_train[:2][:50]

array([[   2, 2452, 4368, ...,    0,    0,    0],
       [   1, 4146,   35, ...,    0,    0,    0]], dtype=int32)

In [111]:
X_train.shape

(14559, 5000)

In [140]:
X_train[:2]

array([[   2, 2452, 4368, ...,    0,    0,    0],
       [   1, 4146,   35, ...,    0,    0,    0]], dtype=int32)

In [128]:
X_train.shape

(14559, 5000)

In [ ]:
embedding_dim = 50
activation = 'relu'
batch_size = 128

model = Sequential()
# model.add(Embedding(input_dim=vocab_size, 
#                            output_dim=embedding_dim, 
#                            input_length=maxlen))
# model.add(Flatten())
model.add(Dense(maxlen, input_dim=maxlen, activation=activation))
model.add(BatchNormalization())
model.add(Dense(2500, activation=activation))
model.add(Dense(1000, activation=activation))
model.add(Dense(250, activation=activation))
model.add(Dense(1000, activation=activation))
model.add(Dense(2500, activation=activation))
model.add(Dense(maxlen, activation=activation))
# model.add(Reshape((5000, 50)))
# model.add(Reshape(((5000))))

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.summary()

early = EarlyStopping(patience=2)

history = model.fit(X_train, X_train,
                    epochs=100,
                    verbose=2,
                    validation_data=(X_test, X_test),
                    batch_size=batch_size,
                    callbacks=[early])


# loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_252 (Dense)            (None, 5000)              25005000  
_________________________________________________________________
batch_normalization_1 (Batch (None, 5000)              20000     
_________________________________________________________________
dense_253 (Dense)            (None, 2500)              12502500  
_________________________________________________________________
dense_254 (Dense)            (None, 1000)              2501000   
_________________________________________________________________
dense_255 (Dense)            (None, 250)               250250    
_________________________________________________________________
dense_256 (Dense)            (None, 1000)              251000    
_________________________________________________________________
dense_257 (Dense)            (None, 2500)              2502500   
__________

In [ ]:
# early = EarlyStopping(patience=2)

# txt = Tokenizer(df['text'].values)
# #txt = pad_sequences(txt, maxlen=500)

# model = Sequential()
# model.add(Embedding(1000, 64))
# model.compile('rmsprop', 'mse')
# X_embedded = model.predict(txt)

# activation = 'relu'

# input_dim = 512
# row_dim = 14559

# input_i = Input(shape=(input_dim, 64))
# encoded_h1 = Dense(256, activation=activation)(input_i)
# encoded_h2 = Dense(128, activation=activation)(encoded_h1)
# latent = Dense(64, activation='tanh')(encoded_h2)
# decoder_h1 = Dense(128, activation=activation)(latent)
# decoder_h2 = Dense(256, activation=activation)(decoder_h1)

# output = Dense(input_dim, activation='tanh')(decoder_h2)

# autoencoder = Model(input_i, output)

# autoencoder.compile('adadelta','mse')
# autencoder.summary()

# #X_embedded = model.predict(np.array(X_train))
# autoencoder.fit(X_embedded, X_embedded, epochs=50,
#             batch_size=256, validation_split=.1, callbacks=[early])


# Make Into Transformer

In [ ]:
from TextCleaner import TextCleaner
help(TextCleaner)

In [10]:
class UniversalSentenceEmbeddingTransformer(TransformerMixin, BaseEstimator):

    def fetch_universal_sentence_embeddings(self, messages, verbose=0):
        """Fetches universal sentence embeddings from Google's
        research paper https://arxiv.org/pdf/1803.11175.pdf.

        INPUTS:
        RETURNS:
        """
        module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

        # Import the Universal Sentence Encoder's TF Hub module
        embed = hub.Module(module_url)

        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_embeddings = session.run(embed(messages))
            embeddings = list()
            for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
                if verbose:
                    print("Message: {}".format(messages[i]))
                    print("Embedding size: {}".format(len(message_embedding)))
                    message_embedding_snippet = ", ".join(
                        (str(x) for x in message_embedding[:3]))
                    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
                embeddings.append(message_embedding)
        return embeddings
    
    def fit(self, X, y=None):
        """interface conforming, and allows use of fit_transform"""
        return self
    
    def transform(self, X):
        return self.fetch_universal_sentence_embeddings(messages=X)

    

In [11]:
from sklearn.model_selection import train_test_split
X, y = df['title'].astype(str).values, df.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=7)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=.5, random_state=7)

In [15]:
# test LinearSVC model as baseline
params = {'clean__lemmatize': [True, False],
          'clean__lower': [True, False],
          'clean__remove_punct': [True, False],
          'clean__extra_punct': [''],
          'clf__C':[1, 2, 3], # 1 best here  [.001, .01, .1, 1, 10, 100, 1000]
          'clf__random_state': [777]}

pipeline = Pipeline([
    ('clean', TextCleaner()),
    ('embed', UniversalSentenceEmbeddingTransformer()),
    ('clf', LinearSVC())
])

grid = GridSearchCV(pipeline, params, cv=5, verbose=1)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


TypeError: must be str, not NoneType

In [ ]:
from TextCleaner import TextCleaner as tc
help(tc)